# Practical Part

Implement a small app that uses all concepts introduced in the theory part.


### Exercise 1: Code Documentation Assistant
Build a system that can analyze a GitHub repository and create comprehensive documentation.

**Requirements:**
1. Create a `GitHubPlugin` that can:
   - Fetch repository contents using GitHub API
   - Extract code files with specific extensions
   - Parse directory structure

2. Create a `CodeAnalysisPlugin` that can:
   - Identify functions and classes
   - Extract comments and docstrings
   - Analyze code complexity
   - Detect dependencies

3. Use semantic memory to:
   - Store analyzed code segments
   - Remember relationships between files
   - Track dependencies
   - Store generated documentation drafts

4. Implement semantic functions for:
   - Explaining code functionality
   - Suggesting improvements
   - Creating usage examples
   - Writing test cases

5. Use the Stepwise Planner to:
   - Plan the documentation structure
   - Prioritize important components
   - Generate documentation in chunks
   - Create a final markdown document

### Exercise 2: Personal Finance Advisor
Create an AI system that can analyze financial data and provide personalized advice.

**Requirements:**
1. Create a `FinancePlugin` that can:
   - Parse CSV bank statements
   - Categorize transactions
   - Calculate spending patterns
   - Track recurring payments

2. Create a `BudgetAnalysisPlugin` that can:
   - Set budget categories
   - Compare spending to budgets
   - Identify unusual expenses
   - Project future spending

3. Use semantic memory to:
   - Store historical spending patterns
   - Remember user preferences
   - Track financial goals
   - Store advice given previously

4. Create semantic functions for:
   - Generating spending insights
   - Providing saving recommendations
   - Creating budget plans
   - Writing financial summaries

5. Use Sequential Planner to:
   - Analyze current financial state
   - Compare with historical data
   - Generate personalized advice
   - Create action plans

**Bonus:** Add integration with a financial news API to provide market context.

### Exercise 3: AI Dungeon Master
Build an intelligent game master for text-based role-playing games.

**Requirements:**
1. Create a `WorldStatePlugin` that can:
   - Track character stats
   - Manage inventory
   - Handle game world state
   - Process player actions

2. Create a `StoryGenerationPlugin` that can:
   - Generate dynamic encounters
   - Create NPCs
   - Design quests
   - Adapt story based on choices

3. Use semantic memory to:
   - Remember player choices
   - Store world history
   - Track NPC relationships
   - Save game state

4. Create semantic functions for:
   - Generating descriptions
   - Creating dialogues
   - Resolving actions
   - Adapting difficulty

5. Use Stepwise Planner to:
   - Plan story arcs
   - Generate responses to actions
   - Manage game balance
   - Create coherent narratives

**Bonus:** Add procedural content generation for locations and items.

In [10]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import KernelArguments
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.core_plugins import TextPlugin
from semantic_kernel.functions import kernel_function, KernelFunctionFromPrompt
from semantic_kernel.planners import SequentialPlanner

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))
text_plugin = kernel.add_plugin(TextPlugin(), "text")

settings = chat_service.get_prompt_execution_settings_class()
settings.temperature = 0.7
settings.top_p = 0.8

kernel.add_service(chat_service)

# Test the kernel
print("Kernel initialized successfully!")

Kernel initialized successfully!


### Let's create an AI Dungeon Master

In [11]:
character_stats_tracker = KernelFunctionFromPrompt(
    function_name="TrackCharacterStats",
    plugin_name="WorldStatePlugin",
    prompt="""
    Track the stats of a character in a story.

    Strength: {{$strength}}
    Dexterity: {{$dexterity}}
    Constitution: {{$constitution}}
    Intelligence: {{$intelligence}}
    Wisdom: {{$wisdom}}
    Charisma: {{$charisma}}
    """,
    description="Track the stats of a character in a story.",
)

inventory_tracker = KernelFunctionFromPrompt(
    function_name="TrackInventory",
    plugin_name="WorldStatePlugin",
    prompt="""
    Track the inventory of a character in a story.

    Inventory: {{$inventory}}
    """,
    description="Track the inventory of a character in a story.",
)

world_state = KernelFunctionFromPrompt(
    function_name="HandleWorldState",
    plugin_name="WorldStatePlugin",
    prompt="""
    Track the state of the world in a story.

    World State: {{$world_state}}
    """,
    description="Track the state of the world in a story.",
)

player_action = KernelFunctionFromPrompt(
    function_name="PlayerAction",
    plugin_name="WorldStatePlugin",
    prompt="""
    Track the action of a player in a DND story.

    Player Action: {{$player_action}}
    """,
    description="Player Action",
)


kernel.add_function(plugin_name="WorldStatePlugin", function=character_stats_tracker)
kernel.add_function(plugin_name="WorldStatePlugin", function=inventory_tracker)
kernel.add_function(plugin_name="WorldStatePlugin", function=world_state)
kernel.add_function(plugin_name="WorldStatePlugin", function=player_action)

encounters = KernelFunctionFromPrompt(
    function_name="Encounters",
    plugin_name="StoryGenerationPlugin",
    prompt="""
    Generate dynamic encounters for the DND story.

    Encounters: {{$encounter}}
    """,
    description="Generate dynamic encounters",
)

create_npc = KernelFunctionFromPrompt(
    function_name="CreateNPC",
    plugin_name="StoryGenerationPlugin",
    prompt="""
    Create an NPC for the story.

    Name: {{$name}}
    Description: {{$description}}
    """,
    description="Create an NPC for the story.",
)

design_quest = KernelFunctionFromPrompt(
    function_name="DesignQuest",
    plugin_name="StoryGenerationPlugin",
    prompt="""
    Design a quest for the story.

    Quest: {{$quest}}
    """,
    description="Design a quest for the story.",
)

adapt_story = KernelFunctionFromPrompt(
    function_name="AdaptStory",
    plugin_name="StoryGenerationPlugin",
    prompt="""
    Adapt the story based on player actions.

    Story: {{$story}}
    """,
    description="Adapt the story based on player actions.",
)

kernel.add_function(plugin_name="StoryGenerationPlugin", function=encounters)
kernel.add_function(plugin_name="StoryGenerationPlugin", function=create_npc)
kernel.add_function(plugin_name="StoryGenerationPlugin", function=design_quest)
kernel.add_function(plugin_name="StoryGenerationPlugin", function=adapt_story)

planner = SequentialPlanner(kernel, service_id)

ask = """Create a DND story introduction and keep track of the world state and character's states. 
Character initial states are Strength: 10, Dexterity: 10, Constitution: 10, Intelligence: 10, Wisdom: 10, Charisma: 10."""

plan = await planner.create_plan(goal=ask)
result = await plan.invoke(kernel)
print("\nFinal Result:")
print(result)


Final Result:
**Title: The Tides of Change**

**Chapter 1: The Calm Before the Storm**

In the year 2023, the world lay in a state of precarious balance. Nations operated on the edge of tranquility, with a trembling pulse beneath the surface. Global leaders spoke of peace in forums and conferences, yet beneath the rhetoric of diplomacy, endless complexities brewed. 

In the bustling city of Aurovia, known for its glistening skyscrapers and rich cultural tapestry, the vibrations of change were all too faintly felt. People moved through the streets, absorbed in their lives, unaware of the undercurrents that defined their existence. Climate change, economic disparities, and technological advancements were shaping a reality that many were yet to comprehend fully.

**Chapter 2: The Echo of Discontent**

As the sun rose over Aurovia, protests erupted in cities across the globe. Activists rallied for climate justice and economic equality, demanding action from their leaders as extreme weathe

### Exercise 5: Research Paper Assistant
Create an AI system that helps with academic research and paper writing.

**Requirements:**
1. Create a `ResearchPlugin` that can:
   - Search academic databases
   - Extract paper contents
   - Parse citations
   - Track references

2. Create a `WritingAssistantPlugin` that can:
   - Generate outlines
   - Structure arguments
   - Format citations
   - Check style guidelines

3. Use semantic memory to:
   - Store paper summaries
   - Remember key findings
   - Track citations
   - Save draft versions

4. Create semantic functions for:
   - Analyzing papers
   - Generating summaries
   - Creating literature reviews
   - Writing sections

5. Use Sequential Planner to:
   - Plan research approach
   - Organize findings
   - Structure paper
   - Generate drafts

**Bonus:** Add LaTeX support for mathematical content.